In [1]:
import numpy as np

In [488]:
class NeuralNet():
    def __init__(self,feat):
        np.random.seed(2)  
        self.weights = {} 
        self.num_layers = 1  
        self.prev_layer = feat
        self.activation_values = {}
        self.delta = {}
        self.partial_deriv = {}
        
    def add_layer(self, num,bias=True):
        self.weights[self.num_layers] =2*(np.random.rand(self.prev_layer+1,num))-1
        self.num_layers += 1
        self.prev_layer=num
    
    def add_bias(self,X):
        bias=np.ones((X.shape[0],1))
        data=np.hstack((bias,X))
        return data
    
    def feed_forward(self,X):
        self.activation_values[1] = self.add_bias(X)
        for layer in range(1,self.num_layers):
            acti = self.sigmoid(np.dot(self.activation_values[layer],self.weights[layer]))
            if layer == self.num_layers-1:
                self.activation_values[layer+1] = acti
            else:
                self.activation_values[layer+1] = self.add_bias(acti)
    
    def test(self,y,y_label):
        new = {}
        new[1] = self.add_bias(y)
        for layer in range(1,self.num_layers):
            acti = self.sigmoid(np.dot(new[layer],self.weights[layer]))
            if layer == self.num_layers-1:
                new[layer+1] = acti
            else:
                new[layer+1] = self.add_bias(acti)
        return new[self.num_layers],np.average((new[self.num_layers]>0.5).astype(int)==y_label)
        
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))
    
    def cost_fn(self,label):
        activation=self.activation_values[self.num_layers]
        return -1*np.average(np.log(activation)*label + np.log(1-activation)*(1-label))
    
    def accuracy(self,labels):
        return np.average((self.activation_values[self.num_layers]>0.5).astype(int)==labels)
        
    
    def sigmoid_der(self,x):
        return x * (1 - x)
    
    def back_propagation(self,label,lbd,train_size):
        self.delta[self.num_layers] = self.activation_values[self.num_layers] - label
        for layer in reversed(range(2, self.num_layers)):
            tmp = (np.dot(self.delta[layer+1],self.weights[layer].T)*self.sigmoid_der(self.activation_values[layer]))
            self.delta[layer] = tmp[:,1:]

        for layer in range(1, self.num_layers):
            theta=self.weights[layer]
            self.partial_deriv[layer] = ((np.dot(self.activation_values[layer].T,self.delta[layer+1])))#+(lbd*theta))/train_size
    
    def gradient_descent(self,learning_rate):
        for layer in range(1, self.num_layers):
            self.weights[layer] = self.weights[layer]-learning_rate*self.partial_deriv[layer]
    
    def train(self,data,label,train_size,learning_rate=0.1,error_limit=0.1,max_epochs=10000,lbd=0):
        i=0
        while True:
            i=i+1
            self.feed_forward(data)
            self.back_propagation(label,train_size,lbd)
            error= self.cost_fn(label)
            self.gradient_descent(learning_rate)
            accuracy = self.accuracy(label)
            if (error<error_limit) | (i==max_epochs):
                return error,i#,self.activation_values[self.num_layers],self.accuracy(label)
        

In [461]:
def mf(a):
    return (a[0]+a[1])


In [462]:
X = 100*(np.random.random((100,2)))+1
y = ((np.apply_along_axis(mf,1,X))>85).astype(int)
y=y.reshape(100,1)

In [489]:
X=np.array([[0,0],[0,1],[1,0],[1,1]])
y=np.array([[0],[1],[1],[0]])
nn=NeuralNet(2)
nn.add_layer(5)
nn.add_layer(3)
nn.add_layer(1)

In [490]:
np.sum(y)

2

In [499]:
for i in [0.001,0.003,0.01,0.03,0.1,0.3,1,3,10]:
    nn=NeuralNet(2)
    nn.add_layer(20)
    nn.add_layer(10)
    nn.add_layer(20)
    nn.add_layer(10)
    nn.add_layer(20)
    nn.add_layer(1)
    print i,nn.train(X,y,4,error_limit=0.1,learning_rate=i)

0.001 (0.6931115082531234, 10000)
0.003 (0.69308314666317961, 10000)
0.01 (0.69289095101380616, 10000)
0.03 (0.099673728143862994, 8929)
0.1 (0.098350091821930843, 2711)
0.3 (0.075565262216721343, 1026)
1 (0.4774613488987316, 10000)
3 (1.3611930400380996, 10000)
10 (4.9995947866543746, 10000)


In [353]:
test_set = 5*(np.random.random((10,1)))+1
test_label = ((np.apply_along_axis(mf,1,test_set))>15).astype(int).reshape(10,1)

In [354]:
nn.test(test_set,test_label)

(array([[ 0.03569607],
        [ 0.91012668],
        [ 0.05795095],
        [ 0.03571448],
        [ 0.91016567],
        [ 0.91015165],
        [ 0.90213275],
        [ 0.03574024],
        [ 0.9084172 ],
        [ 0.03579163]]), 1.0)

In [367]:
nn.weights

{1: array([[ 27.11711866,  -1.23640219,  -0.24969911,  -0.17965665,
          -0.47373616,  12.80500818,  -0.89779544,  -0.88653633,
          -0.78363874],
        [  1.06367457,  -0.99975702,  -5.54792755,  -1.8439967 ,
          -5.08533565,   1.35085649,   3.02354192,   4.65375953,
           3.57086661],
        [ -3.65153391,  -1.74705688,  -5.53135405,  -1.85876602,
          -5.10723617,  -1.39030429,   2.03329033,   4.25741813,
           3.12220041]]),
 2: array([[ -3.51431816,  -1.55297456,  -1.03415756,  -2.63544696,
          -2.48307469,  -1.10781592],
        [ -4.007936  ,  -0.31526887,  -5.31423777,  -0.82818064,
          -1.09393145, -11.62829599],
        [  0.32796078,   0.83413657,  -0.12917808,   0.48573145,
          -0.29226829,  -0.23834581],
        [ -0.73609091,  -0.11281773,   0.67616488,   0.0251413 ,
           0.58535812,  -0.31601318],
        [ -0.75828731,  -0.30117223,   0.9395543 ,  -0.30226743,
          -0.74479342,  -0.61096565],
        [ -0.79

In [453]:
np.zeros((5,2))

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])